# <b>Python for Data Analysis</b>
# 10.  Data Aggregation and Group Operations

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels as sm
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision = 4, suppress = True)
from pandas import Series, DataFrame
%matplotlib inline

## 10.1 How to Think About Group Operations

In [2]:
df = pd.DataFrame({"key1":  ["a", "a", None, "b", "b", "a", None],
                   "key2":  pd.Series([1, 2, 1, 2, 1, None, 1], dtype = "Int64"),
                   "data1": np.random.standard_normal(7),
                   "data2": np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.424734,0.130639
1,a,2,0.090093,0.600729
2,None,1,1.040285,-0.465034
3,b,2,0.441258,1.158618
4,b,1,-0.193280,-0.186538
5,a,<NA>,0.147140,0.161530
6,None,1,0.405954,-2.015022


In [3]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [4]:
grouped.mean()

key1
a   -0.062500
b    0.123989
Name: data1, dtype: float64

In [5]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -0.424734
      2       0.090093
b     1      -0.193280
      2       0.441258
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,1,2
key1,,
a,-0.424734,0.090093
b,-0.193280,0.441258


In [7]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

In [8]:
states

array(['OH', 'CA', 'CA', 'OH', 'OH', 'CA', 'OH'], dtype='<U2')

In [9]:
years

[2005, 2005, 2006, 2005, 2006, 2005, 2006]

In [10]:
df["data1"].groupby([states, years]).mean()

CA  2005    0.118617
    2006    1.040285
OH  2005    0.008262
    2006    0.106337
Name: data1, dtype: float64

In [11]:
df

,key1,key2,data1,data2
0,a,1,-0.424734,0.130639
1,a,2,0.090093,0.600729
2,None,1,1.040285,-0.465034
3,b,2,0.441258,1.158618
4,b,1,-0.193280,-0.186538
5,a,<NA>,0.147140,0.161530
6,None,1,0.405954,-2.015022


In [12]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,-0.062500,0.297633
b,1.5,0.123989,0.486040


In [13]:
df.groupby("key2").mean(numeric_only = True)

,data1,data2
key2,,
1,0.207056,-0.633989
2,0.265675,0.879673


In [14]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.424734  0.130639
     2     0.090093  0.600729
b    1    -0.193280 -0.186538
     2     0.441258  1.158618

In [15]:
df.groupby(["key1", "key2"], dropna = False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [16]:
df.groupby("key1", dropna = False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [17]:
df.groupby(["key1", "key2"], dropna = False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [18]:
df.groupby(["key1", "key2"], dropna = False).count()

data1  data2
key1 key2              
a    1         1      1
     2         1      1
     <NA>      1      1
b    1         1      1
     2         1      1
NaN  1         2      2

In [19]:
df.groupby(["key1", "key2"], dropna = False).count()

data1  data2
key1 key2              
a    1         1      1
     2         1      1
     <NA>      1      1
b    1         1      1
     2         1      1
NaN  1         2      2

In [20]:
df.groupby(["key1", "key2"], dropna = False).count()

data1  data2
key1 key2              
a    1         1      1
     2         1      1
     <NA>      1      1
b    1         1      1
     2         1      1
NaN  1         2      2

### Iterating over Groups

In [21]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -0.424734  0.130639
1    a     2  0.090093  0.600729
5    a  <NA>  0.147140  0.161530
b
  key1  key2     data1     data2
3    b     2  0.441258  1.158618
4    b     1 -0.193280 -0.186538


In [22]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1 -0.424734  0.130639
('a', 2)
  key1  key2     data1     data2
1    a     2  0.090093  0.600729
('b', 1)
  key1  key2    data1     data2
4    b     1 -0.19328 -0.186538
('b', 2)
  key1  key2     data1     data2
3    b     2  0.441258  1.158618


In [23]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,0.441258,1.158618
4,b,1,-0.193280,-0.186538


In [24]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"},
                     axis = "columns")

In [25]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -0.424734  0.130639
1  0.090093  0.600729
2  1.040285 -0.465034
3  0.441258  1.158618
4 -0.193280 -0.186538
5  0.147140  0.161530
6  0.405954 -2.015022
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


### Selecting a Column or Subset of Columns

In [26]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

In [27]:
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

In [28]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.130639
     2     0.600729
b    1    -0.186538
     2     1.158618

In [29]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [30]:
s_grouped.mean()

key1  key2
a     1       0.130639
      2       0.600729
b     1      -0.186538
      2       1.158618
Name: data2, dtype: float64

### Grouping with Dictionaries and Series

In [31]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns = ["a", "b", "c", "d", "e"],
                      index = ["Joe", "Steve", "Wanda", "Jill", "Trey"])
people

,a,b,c,d,e
Joe,1.554965,-0.560994,-0.267312,-0.223265,-0.520791
Steve,0.200718,0.420151,-0.496395,-1.309545,1.217302
Wanda,0.142499,1.663854,0.757987,0.761494,1.025212
Jill,-1.002440,-1.059970,0.290198,-0.693984,-1.578566
Trey,2.115316,1.025977,-0.039335,1.363989,-0.352210


In [32]:
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,1.554965,-0.560994,-0.267312,-0.223265,-0.520791
Steve,0.200718,0.420151,-0.496395,-1.309545,1.217302
Wanda,0.142499,NaN,NaN,0.761494,1.025212
Jill,-1.002440,-1.059970,0.290198,-0.693984,-1.578566
Trey,2.115316,1.025977,-0.039335,1.363989,-0.352210


In [33]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [34]:
by_column = people.groupby(mapping, axis = "columns")
by_column.sum()

,blue,red
Joe,-0.490577,0.473180
Steve,-1.805940,1.838171
Wanda,0.761494,1.167712
Jill,-0.403786,-3.640977
Trey,1.324654,2.789083


In [35]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [36]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions

In [37]:
people

,a,b,c,d,e
Joe,1.554965,-0.560994,-0.267312,-0.223265,-0.520791
Steve,0.200718,0.420151,-0.496395,-1.309545,1.217302
Wanda,0.142499,NaN,NaN,0.761494,1.025212
Jill,-1.002440,-1.059970,0.290198,-0.693984,-1.578566
Trey,2.115316,1.025977,-0.039335,1.363989,-0.352210


In [38]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.554965,-0.560994,-0.267312,-0.223265,-0.520791
4,1.112875,-0.033993,0.250863,0.670005,-1.930777
5,0.343217,0.420151,-0.496395,-0.548051,2.242515


In [39]:
key_list = ["one", "one", "one", "two", "two"]

In [40]:
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,1.554965,-0.560994,-0.267312,-0.223265,-0.520791
4,two,-1.002440,-1.059970,-0.039335,-0.693984,-1.578566
5,one,0.142499,0.420151,-0.496395,-1.309545,1.025212


### Grouping by Index Levels

In [41]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                     [1, 3, 5, 1, 3]],
                                    names = ["cty", "tenor"])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [42]:
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), 
                       columns = columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      2.251901 -0.305850 -0.027610  0.377198  0.983827
1     -1.050697  1.731854  1.031904  0.037035  1.085804
2      0.362981  0.689332  1.824412  2.123286  0.240978
3      0.806779 -0.276175 -1.285643 -1.026886 -0.484479

In [43]:
hier_df.groupby(level = "cty", axis = 1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

In [44]:
df

,key1,key2,data1,data2
0,a,1,-0.424734,0.130639
1,a,2,0.090093,0.600729
2,None,1,1.040285,-0.465034
3,b,2,0.441258,1.158618
4,b,1,-0.193280,-0.186538
5,a,<NA>,0.147140,0.161530
6,None,1,0.405954,-2.015022


In [45]:
grouped = df.groupby("key1")
grouped

In [46]:
grouped["data1"].nsmallest(2)

key1   
a     0   -0.424734
      1    0.090093
b     4   -0.193280
      3    0.441258
Name: data1, dtype: float64

In [47]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [48]:
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,0.571874,0.470090
b,1,0.634537,1.345155


In [49]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0 -0.062500  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.123989  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.118617  0.147140   3.0  0.297633  0.262943  0.130639  0.146085   
b     0.282623  0.441258   2.0  0.486040  0.951168 -0.186538  0.149751   

                                   
          50%       75%       max  
key1                               
a     0.16153  0.381130  0.600729  
b     0.48604  0.822329  1.158618  

[2 rows x 24 columns]

### Column-Wise and Multiple Function Application

In [50]:
tips = pd.read_csv("examples/tips.csv")

In [51]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [52]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [53]:
grouped = tips.groupby(["day", "smoker"])

In [54]:
grouped_pct = grouped["tip_pct"]

In [55]:
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [56]:
tips.groupby(["day", "smoker"])["tip_pct"].agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [57]:
tips.groupby(["day", "smoker"])["tip_pct"].mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [58]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [59]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)]) # (column name, function)

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [60]:
grouped_pct.agg([("average", "mean"), ("stdev", "std")]) # (column name, function)

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [61]:
grouped_pct.agg(["mean", "std", ("Max/min diff.", peak_to_peak)])

mean       std  Max/min diff.
day  smoker                                   
Fri  No      0.151650  0.028123       0.067349
     Yes     0.174783  0.051293       0.159925
Sat  No      0.158048  0.039767       0.235193
     Yes     0.147906  0.061375       0.290095
Sun  No      0.160113  0.042347       0.193226
     Yes     0.187250  0.154134       0.644685
Thur No      0.160298  0.038774       0.193350
     Yes     0.163863  0.039389       0.151240

In [62]:
functions = ["count", "mean", "max"]

In [63]:
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [64]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [65]:
ftuples = [("Average", "mean"), ("Variance", np.var)]

In [66]:
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [67]:
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [68]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
             "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [69]:
grouped = tips.groupby(["day", "smoker"], as_index = False)
grouped.mean(numeric_only = True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [70]:
result_test = grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
                           "size": "sum"})
result_test.reset_index()

index   day smoker   tip_pct                               size
                           min       max      mean       std  sum
0     0   Fri     No  0.120385  0.187735  0.151650  0.028123    9
1     1   Fri    Yes  0.103555  0.263480  0.174783  0.051293   31
2     2   Sat     No  0.056797  0.291990  0.158048  0.039767  115
3     3   Sat    Yes  0.035638  0.325733  0.147906  0.061375  104
4     4   Sun     No  0.059447  0.252672  0.160113  0.042347  167
5     5   Sun    Yes  0.065660  0.710345  0.187250  0.154134   49
6     6  Thur     No  0.072961  0.266312  0.160298  0.038774  112
7     7  Thur    Yes  0.090014  0.241255  0.163863  0.039389   40

## 10.3 Apply: General split-apply-combine

In [71]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [72]:
def top(df, n = 5, column = "tip_pct"):
    return df.sort_values(column, ascending = False)[:n]

In [73]:
top(tips, n = 6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [74]:
tips.sort_values("tip", ascending = False)[:5]

,total_bill,tip,smoker,day,time,size,tip_pct
170,50.81,10.00,Yes,Sat,Dinner,3,0.196812
212,48.33,9.00,No,Sat,Dinner,4,0.186220
23,39.42,7.58,No,Sat,Dinner,4,0.192288
59,48.27,6.73,No,Sat,Dinner,4,0.139424
141,34.30,6.70,No,Thur,Lunch,6,0.195335


In [75]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [76]:
tips.groupby(["smoker", "day"]).apply(top, n = 1, column = "total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [77]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [78]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

This code is alternative to <code>.describe()</code>:

In [79]:
# def f(group):
#    return group.describe()

# grouped.apply(f)

### Suppressing the Group Keys

In [80]:
tips.groupby("smoker", group_keys = False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [81]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

### Quantile and Bucket Analysis

In [82]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,1.698237,0.224865
1,-1.212174,-0.456939
2,1.614577,0.335902
3,-0.641409,1.004635
4,1.077557,1.736898


In [83]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0      (1.628, 3.249]
1    (-1.614, 0.0071]
2     (0.0071, 1.628]
3    (-1.614, 0.0071]
4     (0.0071, 1.628]
5    (-1.614, 0.0071]
6    (-1.614, 0.0071]
7     (0.0071, 1.628]
8     (0.0071, 1.628]
9    (-1.614, 0.0071]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.242, -1.614] < (-1.614, 0.0071] < (0.0071, 1.628] < (1.628, 3.249]]

In [84]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
         "count": group.count(), "mean": group.mean()})

In [85]:
grouped = frame.groupby(quartiles)

In [86]:
grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-3.242, -1.614] data1 -3.235111 -1.614931     55 -1.996026
                 data2 -2.490326  2.364352     55  0.011612
(-1.614, 0.0071] data1 -1.613839  0.005201    430 -0.621445
                 data2 -3.533225  3.159568    430  0.064140
(0.0071, 1.628]  data1  0.007933  1.614577    459  0.665331
                 data2 -3.555997  3.188250    459 -0.017321
(1.628, 3.249]   data1  1.628630  3.249310     56  1.985036
                 data2 -1.710577  2.919030     56 -0.049486

In [87]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-3.242, -1.614] -3.235111 -1.614931    55 -1.996026 -2.490326  2.364352   
(-1.614, 0.0071] -1.613839  0.005201   430 -0.621445 -3.533225  3.159568   
(0.0071, 1.628]   0.007933  1.614577   459  0.665331 -3.555997  3.188250   
(1.628, 3.249]    1.628630  3.249310    56  1.985036 -1.710577  2.919030   

                                  
                 count      mean  
data1                             
(-3.242, -1.614]    55  0.011612  
(-1.614, 0.0071]   430  0.064140  
(0.0071, 1.628]    459 -0.017321  
(1.628, 3.249]      56 -0.049486

In [88]:
quartiles_samp = pd.qcut(frame["data1"], 4)
quartiles_samp.head()

0                   (0.721, 3.249]
1    (-3.2359999999999998, -0.617]
2                   (0.721, 3.249]
3    (-3.2359999999999998, -0.617]
4                   (0.721, 3.249]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.2359999999999998, -0.617] < (-0.617, 0.0415] < (0.0415, 0.721] < (0.721, 3.249]]

In [89]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                                                   
(-3.2359999999999998, -0.617] data1 -3.235111 -0.618701    250 -1.238341
                              data2 -2.789424  2.602040    250 -0.012847
(-0.617, 0.0415]              data1 -0.615873  0.033125    250 -0.268444
                              data2 -3.533225  3.159568    250  0.135283
(0.0415, 0.721]               data1  0.049874  0.720851    250  0.374212
                              data2 -3.555997  2.548791    250 -0.017386
(0.721, 3.249]                data1  0.721999  3.249310    250  1.290759
                              data2 -3.026755  3.188250    250 -0.035062

In [90]:
quartiles_samp = pd.qcut(frame["data1"], 4)
quartiles_samp.head()

0                   (0.721, 3.249]
1    (-3.2359999999999998, -0.617]
2                   (0.721, 3.249]
3    (-3.2359999999999998, -0.617]
4                   (0.721, 3.249]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.2359999999999998, -0.617] < (-0.617, 0.0415] < (0.0415, 0.721] < (0.721, 3.249]]

### Example: Filling Missing Values with Group-Specific Values

In [91]:
s = pd.Series(np.random.standard_normal(6))
s

0    1.886160
1   -1.011962
2   -0.256620
3   -0.422010
4   -0.692683
5   -0.902657
dtype: float64

In [92]:
s[::2] = np.nan
s

0         NaN
1   -1.011962
2         NaN
3   -0.422010
4         NaN
5   -0.902657
dtype: float64

In [93]:
s.fillna(s.mean())

0   -0.778876
1   -1.011962
2   -0.778876
3   -0.422010
4   -0.778876
5   -0.902657
dtype: float64

In [94]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]

group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]

In [95]:
data = pd.Series(np.random.standard_normal(8), index = states)
data

Ohio          0.196363
New York     -0.234778
Vermont      -0.153237
Florida      -0.691903
Oregon       -0.809963
Nevada        0.219700
California    0.616353
Idaho        -1.757577
dtype: float64

In [96]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio          0.196363
New York     -0.234778
Vermont            NaN
Florida      -0.691903
Oregon       -0.809963
Nevada             NaN
California    0.616353
Idaho              NaN
dtype: float64

In [97]:
data.groupby(group_key).mean()

East   -0.243439
West   -0.096805
dtype: float64

In [98]:
data.groupby(group_key).mean()

East   -0.243439
West   -0.096805
dtype: float64

In [99]:
data.groupby(group_key).mean()

East   -0.243439
West   -0.096805
dtype: float64

In [100]:
def fill_mean(group):
    return group.fillna(group.mean())

In [101]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.196363
New York     -0.234778
Vermont      -0.243439
Florida      -0.691903
Oregon       -0.809963
Nevada       -0.096805
California    0.616353
Idaho        -0.096805
dtype: float64

In [102]:
fill_values = {"East": 0.5, "West": -1}

In [103]:
def fill_func(group):
    return group.fillna(fill_values[group.name])

In [104]:
data.groupby(group_key).apply(fill_func)

Ohio          0.196363
New York     -0.234778
Vermont       0.500000
Florida      -0.691903
Oregon       -0.809963
Nevada       -1.000000
California    0.616353
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [105]:
suits = list("HSCD")
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + list("JKQ")
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index = cards)

In [106]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
       ..
9D      9
10D    10
JD     10
KD     10
QD     10
Length: 52, dtype: int64

In [107]:
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [108]:
def draw(deck, n = 5):
    return deck.sample(n) # return a n-number of values from a df

In [109]:
draw(deck)

KD    10
QH    10
3D     3
4H     4
2D     2
dtype: int64

In [110]:
deck.sample(10)

5S      5
QD     10
4H      4
7C      7
2S      2
9H      9
10C    10
3H      3
QH     10
6H      6
dtype: int64

In [111]:
def get_suit(card):
    # last letter is suit
    return card[-1]

In [112]:
deck.groupby(get_suit).apply(draw, n = 2)

C  10C    10
   AC      1
D  3D      3
   5D      5
H  2H      2
   9H      9
S  AS      1
   QS     10
dtype: int64

In [113]:
deck.groupby(get_suit, group_keys = False).apply(draw, n = 2)

KC     10
10C    10
6D      6
2D      2
2H      2
8H      8
2S      2
4S      4
dtype: int64

### Example: Group Weighted Average and Correlation

In [114]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size = 8)})
df

,category,data,weights
0,a,-1.022630,0.019085
1,a,-1.199223,0.589399
2,a,0.117927,0.522916
3,a,-0.603700,0.391013
4,b,-2.169232,0.533516
5,b,-0.765700,0.894136
6,b,1.432308,0.302565
7,b,-0.355516,0.079035


In [115]:
grouped = df.groupby("category")

In [116]:
def get_wavg(group):
    return np.average(group["data"], weights = group["weights"])

In [117]:
grouped.apply(get_wavg)

category
a   -0.591644
b   -0.794081
dtype: float64

In [118]:
get_wavg(df[df.category == "b"])

-0.7940807232149629

In [119]:
get_wavg(df[df.category == "b"])

-0.7940807232149629

In [120]:
get_wavg(df[df.category == "b"])

-0.7940807232149629

In [121]:
close_px = pd.read_csv("examples/stock_px.csv",
                       parse_dates = True,
                       index_col = 0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [122]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [123]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [124]:
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [125]:
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [126]:
rets = close_px.pct_change().dropna()
rets.head()

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386


In [127]:
def get_year(x):
    return x.year

In [128]:
by_year = rets.groupby(get_year)

In [129]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [130]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])

In [131]:
by_year.apply(corr_aapl_msft)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In [132]:
import statsmodels.api as sm

def regress(data, yvar = None, xvars = None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [133]:
by_year.apply(regress, yvar = "AAPL", xvars = ["SPX"])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 Group Transforms and "Unwrapped" GroupBys

In [134]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [135]:
g = df.groupby("key")["value"]
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [136]:
def get_mean(group):
    return group.mean()

In [137]:
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [138]:
g.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [139]:
def times_two(group):
    return group * 2

In [140]:
g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [141]:
def get_ranks(group):
    return group.rank(ascending = False)

In [142]:
g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [143]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [144]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [145]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [146]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [147]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

## 10.5 Pivot Tables and Cross-Tabulation

In [148]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [150]:
tips.pivot_table(index = ["day", "smoker"],
                 values = ["size", "tip", "tip_pct", "total_bill"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [152]:
tips.pivot_table(index = ["time", "day"],
                 columns = "smoker",
                 values = ["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [152]:
tips.pivot_table(index = ["time", "day"],
                 columns = "smoker",
                 values = ["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [153]:
tips.pivot_table(index = ["time", "day"],
                 columns = "smoker",
                 values = ["tip_pct", "size"],
                 margins = True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [155]:
tips.pivot_table(index = ["time", "smoker"],
                 columns = "day",
                 values = "tip_pct",
                 aggfunc = len,
                 margins = True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [156]:
tips.pivot_table(index = ["time", "smoker"],
                 columns = "day",
                 values = "tip_pct",
                 aggfunc = len,
                 margins = True,
                 fill_value = 0)

day            Fri  Sat  Sun  Thur  All
time   smoker                          
Dinner No        3   45   57     1  106
       Yes       9   42   19     0   70
Lunch  No        1    0    0    44   45
       Yes       6    0    0    17   23
All             19   87   76    62  244

In [157]:
tips.pivot_table(index = ["time", "size", "smoker"],
                 columns = "day",
                 values = "tip_pct",
                 fill_value = 0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
...                      ...       ...       ...       ...
Lunch  3    Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

[21 rows x 4 columns]

### Cross-Tabulations: Crosstab

In [162]:
from io import StringIO

In [163]:
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

In [164]:
data = pd.read_table(StringIO(data), sep = "\s+")
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [165]:
pd.crosstab(data["Nationality"], data["Handedness"], margins = True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [166]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [173]:
pd.crosstab([tips["time"], tips["day"]], 
            tips["smoker"], 
            margins = True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [172]:
tips.pivot_table(index = ["time", "day"], values = "size", 
                 columns = "smoker", aggfunc = len,
                 margins = True, fill_value = 0)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244